In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('C:/Users/Ndagano/Desktop/python_pjcts/pytorch/iris_flowers/data/iris.csv')

In [3]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [4]:
data['species'].value_counts()

species
setosa        50
versicolor    50
virginica     50
Name: count, dtype: int64

In [9]:
x = data.drop('species', axis=1)
y = data['species']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42) 
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

print("Train:", X_train.shape, y_train.shape)
print("Val  :", X_val.shape, y_val.shape)
print("Test :", X_test.shape, y_test.shape)

Train: (120, 4) (120,)
Val  : (15, 4) (15,)
Test : (15, 4) (15,)


In [21]:
encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_val_enc = encoder.transform(y_val)
y_test_enc = encoder.transform(y_test)

In [22]:
class dataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [25]:
train_dataset = dataset(torch.tensor(X_train.values, dtype=torch.float32), torch.tensor(y_train_enc, dtype=torch.float32))
val_dataset = dataset(torch.tensor(X_val.values, dtype=torch.float32), torch.tensor(y_val_enc, dtype=torch.float32))
test_dataset = dataset(torch.tensor(X_test.values, dtype=torch.float32), torch.tensor(y_test_enc, dtype=torch.float32))

In [26]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [28]:
class IrisModel(nn.Module):
    def __init__(self):
        super(IrisModel, self).__init__()
        self.fc1 = nn.Linear(4, 16)
        self.fc2 = nn.Linear(16, 3)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return self.softmax(x)

In [29]:
model = IrisModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [31]:
epochs = 20

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    with torch.no_grad():
        model.eval()
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels.long())
            running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')

Epoch [1/20], Loss: 1.0208
Epoch [2/20], Loss: 1.0065
Epoch [3/20], Loss: 1.0026
Epoch [4/20], Loss: 0.9889
Epoch [5/20], Loss: 0.9875
Epoch [6/20], Loss: 0.9877
Epoch [7/20], Loss: 0.9832
Epoch [8/20], Loss: 0.9765
Epoch [9/20], Loss: 0.9650
Epoch [10/20], Loss: 0.9643
Epoch [11/20], Loss: 0.9567
Epoch [12/20], Loss: 0.9690
Epoch [13/20], Loss: 0.9545
Epoch [14/20], Loss: 0.9465
Epoch [15/20], Loss: 0.9491
Epoch [16/20], Loss: 0.9440
Epoch [17/20], Loss: 0.9335
Epoch [18/20], Loss: 0.9339
Epoch [19/20], Loss: 0.9359
Epoch [20/20], Loss: 0.9299


In [32]:
with torch.no_grad():
    model.eval()
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.long()).sum().item()

    print(f'Accuracy of the model on the test set: {100 * correct / total:.2f}%')

Accuracy of the model on the test set: 60.00%


In [35]:
def predict(model, x):
    model.eval()
    with torch.no_grad():
        x_tensor = torch.tensor(x, dtype=torch.float32).unsqueeze(0)
        outputs = model(x_tensor)
        _, predicted = torch.max(outputs.data, 1)
        return encoder.inverse_transform(predicted.numpy())

In [36]:
predict(model=model, x=[5.1, 3.5, 1.4, 0.2])  # Example input for prediction

array(['setosa'], dtype=object)